In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow -U tensorflow_decision_forests > /dev/null

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow_decision_forests as tfdf
from sklearn.metrics import classification_report
base_dir = '/content/drive/Shareddrives/datasci/ton_iot/result/'

In [ ]:
X = []
Y = None
with open(base_dir+'ALL.csv','r') as infile:
  for line in infile:
    pred = line.rstrip().split(',')
    if pred[0] == 'LABEL':
      Y = np.array(pred[1:], dtype='int8').transpose()
    else:
      X.append(pred[1:])
X = np.array(X, dtype='int8').transpose()

In [ ]:
model = tfdf.keras.RandomForestModel()
model.compile()

Use /tmp/tmps1pecb_8 as temporary training directory


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size=0.3,
                                                    random_state=23)

In [ ]:
history = model.fit(X_train,
                    Y_train,
                    verbose = 1)

In [ ]:
predict = model.predict(X_test, verbose=1)

1297/1297 [==============================] - 3s 2ms/step


In [ ]:
tp,tn,fp,fn = 0,0,0,0
predictn = predict.flatten().round().tolist()
len(predictn)
for i in range(len(predictn)):
  if predictn[i]==1 and Y_test[i]==1:
    tp+=1
  elif predictn[i]==0 and Y_test[i]==0:
    tn+=1
  elif predictn[i]==0 and Y_test[i]==1:
    fp+=1
  elif predictn[i]==1 and Y_test[i]==0:
    fn+=1
    
print(tp,tn,fp,fn)
classification_report(Y_test, predictn, output_dict=True)

14583 26763 48 100


{'0': {'f1-score': 0.9972426128106718,
  'precision': 0.9982096900525903,
  'recall': 0.9962774075866433,
  'support': 26863},
 '1': {'f1-score': 0.9949512178481271,
  'precision': 0.9931894027106177,
  'recall': 0.9967192946483494,
  'support': 14631},
 'accuracy': 0.996433219260616,
 'macro avg': {'f1-score': 0.9960969153293995,
  'precision': 0.9956995463816041,
  'recall': 0.9964983511174963,
  'support': 41494},
 'weighted avg': {'f1-score': 0.9964346550409463,
  'precision': 0.9964395106507394,
  'recall': 0.996433219260616,
  'support': 41494}}

In [ ]:
print(model.summary())

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (10):
	data:0.0
	data:0.1
	data:0.2
	data:0.3
	data:0.4
	data:0.5
	data:0.6
	data:0.7
	data:0.8
	data:0.9

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.  "__LABEL"  6.041529 ################
    2. "data:0.4"  5.410826 #############
    3. "data:0.5"  5.314544 #############
    4. "data:0.2"  5.276143 #############
    5. "data:0.9"  4.265942 ##########
    6. "data:0.1"  4.093667 #########
    7. "data:0.3"  3.802675 ########
    8. "data:0.0"  3.730265 ########
    9. "data:0.6"  3.368675 #######
   10. "data:0.7"  1.976660 ##
   11. "data:0.8"  1.227219 

Variable Importance: NUM_AS_ROOT:
    1. "data:0.8" 154.000000

In [ ]:
model.save('/content/drive/Shareddrives/datasci/ton_iot/result/RF.tf')